In [22]:
from tqdm.auto import tqdm

In [40]:
from pymystem3 import Mystem

mystem = Mystem()


In [2]:
def ru_lemmatize(line):
    lemmas = mystem.lemmatize(line)
    return ''.join(lemmas)[:-1]
ru_lemmatize("Красивая мама красиво мыла раму")

'красивый мама красиво мыть рама'

In [3]:
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
lemmatizer = FrenchLefffLemmatizer()


In [5]:
def fr_lemmatize(word):
    lemma = lemmatizer.lemmatize(word)
    return lemma
fr_lemmatize("voitures")

'voiture'

In [15]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

In [36]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
def en_lemmatize(line):
    pos_tagged_ngramm = pos_tag(line.split())
    lemmatized_line_list = []
    for word_el in pos_tagged_ngramm:
        pos = get_wordnet_pos(word_el[1])
        if pos:
            lemma = lemmatizer.lemmatize(word_el[0], pos =pos)
        else:
            lemma = word_el[0]
        lemmatized_line_list.append(lemma)
    return ' '.join(lemmatized_line_list)

preprocess_words("cheerily", en_lemmatize)

'cheerily'

In [10]:
def preprocess_words(words_string, lemmatizer_func):
    preprocessed_words = []
    for word in words_string.split():
        lemma = lemmatizer_func(word.lower())
        preprocessed_words.append(lemma)
    return ' '.join(preprocessed_words)
preprocess_words("les voitures", fr_lemmatize)

'les voiture'

In [17]:
import pandas as pd

In [49]:
three_steps_with_wordforms = pd.read_csv("./reverse_connected_dicts/to_release/three_steps_with_wordforms.csv")

In [19]:
three_steps_with_wordforms.head()

,en,ru,fr,vote
0,gaily,весело,joyeusement,2
1,gaily,ярко,no_equality,2
2,merrily,весело,joyeusement,2
3,merrily,счастливо,joyeusement,2
4,merrily,тонет,no_equality,1


In [30]:
def et_lemma_list(orig_word_row,  lemmat_funct):
    lemma_dict = {}
    for orig_word in tqdm(list(set(orig_word_row))):
        lemma = preprocess_words(orig_word, lemmat_funct)
        lemma_dict[orig_word] = lemma
    expand_lemmas = []
    for orig_word_n_unique in list(orig_word_row):
        expand_lemmas.append(lemma_dict[orig_word_n_unique])  
    return expand_lemmas

In [31]:
eng_lemmas= et_lemma_list(three_steps_with_wordforms['en'], en_lemmatize)

In [32]:
len(three_steps_with_wordforms), len(eng_lemmas)

(670002, 670002)

In [50]:
three_steps_with_wordforms['en_lemmas'] = eng_lemmas
three_steps_with_wordforms.head()

,en,ru,fr,vote,en_lemmas
0,gaily,весело,joyeusement,2,gaily
1,gaily,ярко,no_equality,2,gaily
2,merrily,весело,joyeusement,2,merrily
3,merrily,счастливо,joyeusement,2,merrily
4,merrily,тонет,no_equality,1,merrily


In [51]:
ru_lemmas= et_lemma_list(three_steps_with_wordforms['ru'], ru_lemmatize)
fr_lemmas= et_lemma_list(three_steps_with_wordforms['fr'], fr_lemmatize)

In [46]:
list(three_steps_with_wordforms.columns.values)

['en', 'ru', 'fr', 'vote', 'en_lemmas', 'ru_lemmas', 'fr_lemmas']

In [54]:
three_steps_with_wordforms['ru_lemmas'] = ru_lemmas
three_steps_with_wordforms['fr_lemmas'] = fr_lemmas
three_steps_with_wordforms.head()

,en,ru,fr,vote,en_lemmas,ru_lemmas,fr_lemmas
0,gaily,весело,joyeusement,2,gaily,весело,joyeusement
1,gaily,ярко,no_equality,2,gaily,ярко,no_equality
2,merrily,весело,joyeusement,2,merrily,весело,joyeusement
3,merrily,счастливо,joyeusement,2,merrily,счастливо,joyeusement
4,merrily,тонет,no_equality,1,merrily,тонуть,no_equality


In [56]:
new_order = ['en', 'en_lemmas','ru', 'ru_lemmas','fr','fr_lemmas', 'vote']

In [57]:
three_steps_with_wordforms = three_steps_with_wordforms.ix[:,new_order]
three_steps_with_wordforms.head()

/Users/nigula/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


,en,en_lemmas,ru,ru_lemmas,fr,fr_lemmas,vote
0,gaily,gaily,весело,весело,joyeusement,joyeusement,2
1,gaily,gaily,ярко,ярко,no_equality,no_equality,2
2,merrily,merrily,весело,весело,joyeusement,joyeusement,2
3,merrily,merrily,счастливо,счастливо,joyeusement,joyeusement,2
4,merrily,merrily,тонет,тонуть,no_equality,no_equality,1


In [58]:
three_steps_with_wordforms.to_csv("./reverse_connected_dicts/to_release/three_steps_with_wordforms.csv", index = False)

In [76]:
three_steps_with_wordforms_f = three_steps_with_wordforms.iloc[:200000]

In [78]:
len(three_steps_with_wordforms_f)

200000

In [79]:
three_steps_with_wordforms_f.to_csv("./reverse_connected_dicts/to_release/three_steps_with_wordforms_20k.csv", index = False)

In [80]:
three_steps_with_wordforms_f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 7 columns):
en           200000 non-null object
en_lemmas    200000 non-null object
ru           200000 non-null object
ru_lemmas    200000 non-null object
fr           200000 non-null object
fr_lemmas    200000 non-null object
vote         200000 non-null int64
dtypes: int64(1), object(6)
memory usage: 10.7+ MB


In [86]:
three_steps_with_wordforms_20_40k = three_steps_with_wordforms.iloc[200000:400000]
three_steps_with_wordforms_20_40k.to_csv("./reverse_connected_dicts/to_release/three_steps_with_wordforms_20_40k.csv", index = False)

In [87]:
three_steps_with_wordforms_20_40k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 200000 to 399999
Data columns (total 7 columns):
en           200000 non-null object
en_lemmas    200000 non-null object
ru           200000 non-null object
ru_lemmas    200000 non-null object
fr           200000 non-null object
fr_lemmas    200000 non-null object
vote         200000 non-null int64
dtypes: int64(1), object(6)
memory usage: 10.7+ MB


In [88]:
three_steps_with_wordforms_40k_pl = three_steps_with_wordforms.iloc[400000:600000]
three_steps_with_wordforms_40k_pl.to_csv("./reverse_connected_dicts/to_release/three_steps_with_wordforms_40k_pl.csv", index = False)

In [89]:
three_steps_with_wordforms_last = three_steps_with_wordforms.iloc[600000:]
three_steps_with_wordforms_last.to_csv("./reverse_connected_dicts/to_release/three_steps_with_wordforms_last.csv", index = False)

In [84]:
three_steps_with_wordforms_40k_pl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270002 entries, 400000 to 670001
Data columns (total 7 columns):
en           270002 non-null object
en_lemmas    270002 non-null object
ru           270002 non-null object
ru_lemmas    270002 non-null object
fr           270002 non-null object
fr_lemmas    270002 non-null object
vote         270002 non-null int64
dtypes: int64(1), object(6)
memory usage: 14.4+ MB


In [93]:
a = set()
a.add(1)
a.add(1)
a.add(2)
a

{1, 2}

In [ ]:
en_lemmas = []
for en_word in tqdm():
    lemma = 